# BATTLE OF THE NEIGHBORHOODS
### Where to Live in America? Low Cost, Diverse Food, and Less Serial Killers

## 1. INTRODUCTION

The United States covers 3.797 million square miles across North America.  As of 2020, there is an estimated 331.3 million people living within its 50 states.  Each of the 50 states has its own unique characteristics.  Some states, like New York, are home to booming metropolitan areas.  Some hold beautiful vast national parks, like Arizona.  Others still have the quiet rustic farmlands, like Oklahoma.  Some states, such as California, contain a bit of everything.

Deciding where to move can be difficult.  There are various factors to consider; jobs, housing, schools, etc.  For this capstone project, I will dive into three:

* Cost of living
* Variety of food choices available
* The likelihood of coming face to face with a serial killer

Yes, those are three very different factors for one to consider.  However, together, I believe they can make a more compelling case for states that do not receive as much attention.  

My target audience is people looking for an inexpensive state to live, with a lot of food options, and do not want to face off with a serial killer!

## 2. DATA ACQUISITION

For my analysis, I will be using two data sources:

* worldpopulationreview.com 
* Foursquare

__2.1 Foursquare__

Utilizing the Foursquare API, I will obtain a sample of 50 different types of restaurants available per state.  I will then analyze and compare each state to determine the level of diversity.  For example, if one state has 20 types of restaurants in the sample of 50, they are more diverse than a state that only has 5 types of restaurants in the sample of 50.

__2.2 worldpopulationreview.com (Cost of Living by State)__

https://worldpopulationreview.com/states/cost-of-living-index-by-state/

I will utilize the general cost of living index in order to compare the 50 states.  The average general cost of living index in the United States is 100.  Each state falls on a range between 86 and 193.  Virginia, for example, is a 100.7, which is right about average.

This data also contains other more specific cost of living measurements, such as utilities and transportation.  However, we will not be analyzing those variables in this project.

__2.3 worldpopulationreview.com (Serial Killers by State)__

https://worldpopulationreview.com/states/serial-killers-by-state/

This data contains the number of (known) serial killer victims in each state.  It also contains the total population of each state (as of 2020).  Based on the information provided, I will calculate the probability of encountering a serial killer in every state.  For example, Virginia has 238 confirmed serial killer victims and a population of 8,626,207.  Therefore, if you live in that state, you have a 0.0028% chance of a serial killer encounter.

In [ ]:
## 3. Data Analysis

In [ ]:
## 4. Conclusion